In [40]:
import pandas as pd
import pymysql
import numpy as np
import requests
from datetime import datetime, timedelta
import time
from pycoingecko import CoinGeckoAPI

# CoinMarketCap.com API Key
api_key = "a86e3326-8c8d-434e-897a-5d6becbf1f35"
top_symbols_count = 1300
required_tags = ['Binance Smart Chain Ecosystem', 'Polkadot Ecosystem', 'Cosmos Ecosystem', 'Solana Ecosystem', 'Avalanche Ecosystem', 'HECO Chain Ecosystem', 'Layer 1', 'Decentralized Finance (DeFi)', 'Yield Farming', 'Decentralized Exchange Token (DEX)', 'Governance', 'DaoMaker Ecosystem', 'Gaming', 'Stablecoins', 'Asset Manager', 'Centralized Exchange Token (CEX)', 'Eth 2.0 Staking', 'Yield Aggregator', 'Lending/Borrowing', 'Options', 'Launchpad']
required_symbols = ['JOE']
cg = CoinGeckoAPI()

In [41]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()

### Today's Top Coins (marcketCap) under 1100

In [42]:
def get_market_data(page):
    market_data = cg.get_coins_markets(vs_currency="usd", page=page, per_page=250)
    ids = []
    for data in market_data:
        roi = data['roi']['times'] if data['roi'] else 0
        ids.append(data['symbol'].upper())
    return ids

def get_top_n_coins_symbol(n):
    page = 0
    totla_downloaded = 0
    rows = []
    while totla_downloaded < n:
        page = page + 1
        data = get_market_data(page)
        if not len(data):
            break
        totla_downloaded = totla_downloaded + len(data)
        rows.extend(data)
        time.sleep(1)
    rows = rows[0:n]
    return rows


In [43]:
# top_symbols = get_top_n_coins_symbol(top_symbols_count)

In [44]:
# top_symbols

In [45]:
def get_defi_tvl_data_df():
    defi_tvl = pd.read_sql_query("select * from notebook_defi_tvl;", __connect__())
    defi_tvl = defi_tvl.sort_values(['date', 'Symbol', 'Name'])
    defi_tvl['date'] = pd.to_datetime(defi_tvl['date'].dt.strftime('%Y-%m-%d'), format='%Y-%m-%d')
    defi_tvl = defi_tvl.groupby(['date', 'Symbol', 'Name']).tail(1)
    defi_tvl = defi_tvl.reset_index(drop=True)
    return defi_tvl
    

In [46]:
def get_supply_data_df():
    cmc_supply = pd.read_sql_query("select * from notebook_cmc_supply;", __connect__())
    cmc_supply['TVL'] = np.nan
    cgck_supply = pd.read_sql_query("select * from notebook_coingecko_supply;", __connect__())
    cgck_min_date = cgck_supply.date.min()
    cmc_supply = cmc_supply[cmc_supply['date'] < cgck_min_date]
    cmc_supply = cmc_supply[cmc_supply['Symbol'].isin(cgck_supply.Symbol.unique().tolist())]
    master_df = cmc_supply.append(cgck_supply)
    master_df['TVL Coingecko'] = master_df['TVL']
    del master_df['TVL']
    return master_df

In [47]:
def update_empty_mcap_value(row):
    mcap = row['Marketcap CMC']
    if mcap == 0 or pd.isna(mcap):
        row['Marketcap CMC'] = row['Total Supply CMC'] * row['Price']
    return row

    

In [48]:
def get_all_other_datasource_symbols():
    sql = 'select * from notebook_all_required_symbols_cgk;'
    symbols = pd.read_sql_query(sql, __connect__())
    return symbols

In [49]:
??df.applymap

In [50]:
def get_joins_data_df(all_datasource_symbols):
    # master_symbols = pd.read_sql_query("select symbol as Symbol, current_name as Name, group_concat(tags, ',') as Tags from cgk_coin_metainfo group by symbol, current_name;", __connect__())
    master_symbols = pd.read_sql_query("""select cgk.id as id, cgk.symbol as Symbol, cgk.`name` as Name, group_concat(cgk.tags, ',') as Tags from cgk_coin_metainfo cgk
group by cgk.id, cgk.symbol, cgk.name""", __connect__())
    cmc_supply = get_supply_data_df()
    cmc_supply = cmc_supply.apply(update_empty_mcap_value, axis = 1)
    print(cmc_supply[cmc_supply['Symbol'] == 'JOE'])
    db_tvl = pd.read_sql_query("select * from notebook_debank_data;", __connect__())
    # defi_tvl = get_defi_tvl_data_df()
    dune_tua = pd.read_sql_query("select * from notebook_dune_tua;", __connect__())
    tt_metrics = pd.read_sql_query("select * from notebook_tt_metric_v2;", __connect__())
    defistation_tvl = pd.read_sql_query("select * from notebook_defisation_tvl;", __connect__())
    solana_tvl = pd.read_sql_query("select * from notebook_solana_tvl;", __connect__())
    master_df = pd.merge(cmc_supply, db_tvl, how='left', left_on=['date', 'Symbol', 'Name'], right_on=['date', 'Symbol', 'Name'])
    # master_df = pd.merge(master_df, defi_tvl, how='left', left_on=['date', 'Symbol', 'Name'], right_on=['date', 'Symbol', 'Name'])
    master_df = pd.merge(master_df, dune_tua, how='left', left_on=['date', 'Symbol', 'Name'], right_on=['date', 'Symbol', 'Name'])
    master_df = pd.merge(master_df, tt_metrics, how='left', left_on=['date', 'Symbol', 'Name'], right_on=['date', 'Symbol', 'Name'])
    master_df = pd.merge(master_df, defistation_tvl, how='left', left_on=['date', 'Symbol', 'Name'], right_on=['date', 'Symbol', 'Name'])
    master_df = pd.merge(master_df, solana_tvl, how='left', left_on=['date', 'Symbol', 'Name'], right_on=['date', 'Symbol', 'Name'])
    master_df = pd.merge(master_df, master_symbols, left_on=['Symbol', 'Name'], right_on=['Symbol', 'Name'])
    # master_df['required'] = (master_df['is_defistation'] == 1) | (master_df['is_solana'] == 1) | (master_df['is_debank'] == 1) | (master_df['is_tt_coin'] == 1) | (master_df['is_defi_tvl'] == 1) | (master_df['is_dune_tua'] == 1)
    master_df['required'] = (master_df['Name'].isin(all_datasource_symbols['name_in_coingecko']))
    del master_df['is_debank']
    del master_df['is_tt_coin']
    # del master_df['is_defi_tvl']
    del master_df['is_dune_tua']
    del master_df['is_cmc_supply']
    del master_df['is_defistation']
    del master_df['is_solana']
    master_df = master_df[(master_df['Marketcap CMC'] > 10000000) | (master_df['Symbol'].isin(required_symbols))].reset_index(drop=True)
    return master_df

In [51]:
def get_data_df(all_datasource_symbols):
    df = get_joins_data_df(all_datasource_symbols)
    df = df[(df['Symbol'].isin(all_datasource_symbols)) | (df['required'])]
    df = df.reset_index(drop=True)
    return df

In [52]:
all_data_source_symbols = get_all_other_datasource_symbols()
df = get_data_df(all_data_source_symbols)

            date Symbol Name     Price  Marketcap CMC   Volume CMC  \
21951 2021-08-09    JOE  JOE  0.034519   0.000000e+00    132490.16   
22360 2021-08-10    JOE  JOE  0.034519   0.000000e+00    132490.16   
22769 2021-08-11    JOE  JOE  0.032930   0.000000e+00    108022.29   
23177 2021-08-12    JOE  JOE  0.028150   0.000000e+00    337297.17   
23586 2021-08-13    JOE  JOE  0.028902   0.000000e+00    100933.98   
23994 2021-08-14    JOE  JOE  0.029080   0.000000e+00     87467.90   
24402 2021-08-15    JOE  JOE  0.033736   0.000000e+00    188103.49   
24810 2021-08-16    JOE  JOE  0.034692   0.000000e+00     99282.35   
25219 2021-08-17    JOE  JOE  0.032698   0.000000e+00    132087.83   
25628 2021-08-18    JOE  JOE  0.036597   0.000000e+00    370884.36   
26037 2021-08-19    JOE  JOE  0.064561   0.000000e+00   1038512.68   
26446 2021-08-20    JOE  JOE  0.083500   0.000000e+00   1830644.07   
26855 2021-08-21    JOE  JOE  0.354384   0.000000e+00   5261821.68   
27264 2021-08-22    

In [53]:
len(all_data_source_symbols)

459

In [54]:
df[df['Symbol'] == 'JOE']

,date,Symbol,Name,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Coingecko,...,Revenue Supply Side TT,Revenue Protocol TT,Price/Sale,GMV TT,tvl_defistation,users_defistation,tvl_solana,id,Tags,required
40587,2021-08-09,JOE,JOE,0.034519,0.000000e+00,132490.16,0.0,0.0,0.000000e+00,2.641678e+07,...,0.0,0.0,0.0,1418956.68,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40588,2021-08-10,JOE,JOE,0.034519,0.000000e+00,132490.16,0.0,0.0,0.000000e+00,2.505540e+07,...,0.0,0.0,0.0,1326369.61,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40589,2021-08-11,JOE,JOE,0.032930,0.000000e+00,108022.29,0.0,0.0,0.000000e+00,2.346859e+07,...,0.0,0.0,0.0,971290.74,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40590,2021-08-12,JOE,JOE,0.028150,0.000000e+00,337297.17,0.0,0.0,0.000000e+00,2.313089e+07,...,0.0,0.0,0.0,737123.18,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40591,2021-08-13,JOE,JOE,0.028902,0.000000e+00,100933.98,0.0,0.0,0.000000e+00,2.305233e+07,...,0.0,0.0,0.0,819679.08,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40592,2021-08-14,JOE,JOE,0.029080,0.000000e+00,87467.90,0.0,0.0,0.000000e+00,2.291475e+07,...,0.0,0.0,0.0,599596.18,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40593,2021-08-15,JOE,JOE,0.033736,0.000000e+00,188103.49,0.0,0.0,0.000000e+00,2.076003e+07,...,0.0,0.0,0.0,1533653.16,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40594,2021-08-16,JOE,JOE,0.034692,0.000000e+00,99282.35,0.0,0.0,0.000000e+00,2.434043e+07,...,0.0,0.0,0.0,3723993.98,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40595,2021-08-17,JOE,JOE,0.032698,0.000000e+00,132087.83,0.0,0.0,0.000000e+00,2.433395e+07,...,0.0,0.0,0.0,6427245.27,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40596,2021-08-18,JOE,JOE,0.036597,0.000000e+00,370884.36,0.0,0.0,0.000000e+00,2.673547e+07,...,0.0,0.0,0.0,10880539.18,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True


In [55]:
df2 = df.apply(update_empty_mcap_value, axis = 1)
df2[df2['Symbol'] == 'JOE']

,date,Symbol,Name,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Coingecko,...,Revenue Supply Side TT,Revenue Protocol TT,Price/Sale,GMV TT,tvl_defistation,users_defistation,tvl_solana,id,Tags,required
40587,2021-08-09,JOE,JOE,0.034519,0.000000e+00,132490.16,0.0,0.0,0.000000e+00,2.641678e+07,...,0.0,0.0,0.0,1418956.68,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40588,2021-08-10,JOE,JOE,0.034519,0.000000e+00,132490.16,0.0,0.0,0.000000e+00,2.505540e+07,...,0.0,0.0,0.0,1326369.61,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40589,2021-08-11,JOE,JOE,0.032930,0.000000e+00,108022.29,0.0,0.0,0.000000e+00,2.346859e+07,...,0.0,0.0,0.0,971290.74,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40590,2021-08-12,JOE,JOE,0.028150,0.000000e+00,337297.17,0.0,0.0,0.000000e+00,2.313089e+07,...,0.0,0.0,0.0,737123.18,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40591,2021-08-13,JOE,JOE,0.028902,0.000000e+00,100933.98,0.0,0.0,0.000000e+00,2.305233e+07,...,0.0,0.0,0.0,819679.08,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40592,2021-08-14,JOE,JOE,0.029080,0.000000e+00,87467.90,0.0,0.0,0.000000e+00,2.291475e+07,...,0.0,0.0,0.0,599596.18,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40593,2021-08-15,JOE,JOE,0.033736,0.000000e+00,188103.49,0.0,0.0,0.000000e+00,2.076003e+07,...,0.0,0.0,0.0,1533653.16,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40594,2021-08-16,JOE,JOE,0.034692,0.000000e+00,99282.35,0.0,0.0,0.000000e+00,2.434043e+07,...,0.0,0.0,0.0,3723993.98,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40595,2021-08-17,JOE,JOE,0.032698,0.000000e+00,132087.83,0.0,0.0,0.000000e+00,2.433395e+07,...,0.0,0.0,0.0,6427245.27,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True
40596,2021-08-18,JOE,JOE,0.036597,0.000000e+00,370884.36,0.0,0.0,0.000000e+00,2.673547e+07,...,0.0,0.0,0.0,10880539.18,NaN,NaN,NaN,joe,"Avalanche Ecosystem,",True


In [56]:
df.columns

Index(['date', 'Symbol', 'Name', 'Price', 'Marketcap CMC', 'Volume CMC',
       'Circulating supply CMC', 'MAX Supply CMC', 'Total Supply CMC',
       'TVL Coingecko', 'Users 24hr Debank', '24hr_Debank_users_max',
       'Transactions Count Debank', 'Trading Volume Debank',
       'Borrowing Volume Debank', 'TVL Debank',
       'Users life-to-date duneanlytics', 'Revenue TT',
       'Revenue Supply Side TT', 'Revenue Protocol TT', 'Price/Sale', 'GMV TT',
       'tvl_defistation', 'users_defistation', 'tvl_solana', 'id', 'Tags',
       'required'],
      dtype='object')

In [57]:
df[df['Symbol'] == 'AAVE']['24hr_Debank_users_max'][-30:]

669    7460.0
670    5948.0
671    5964.0
672    5891.0
673    6816.0
674    6162.0
675    6048.0
676    3708.0
677    3533.0
678     764.0
679    2136.0
680    2555.0
681    1458.0
682    1538.0
683    2274.0
684    9369.0
685    8018.0
686    7147.0
687    6028.0
688    7545.0
689    5521.0
690    5781.0
691    6241.0
692    6299.0
693    5745.0
694    5841.0
695    6369.0
696    6100.0
697    5693.0
698    5774.0
Name: 24hr_Debank_users_max, dtype: float64

In [58]:
original_df = df.copy()

In [59]:
len(original_df.Symbol.unique().tolist())

288

In [60]:
len(df.Symbol.unique().tolist())

288

In [61]:
# df.columns

In [62]:
ffill_columns = ['Users life-to-date duneanlytics', 'tvl_defistation', 'tvl_solana', 'tvl_defistation', 'Revenue TT', 'Revenue Supply Side TT', 'Revenue Protocol TT', 'Price/Sale', 'GMV TT']

def ffill_columns_values(df, columns):
    df = df.reset_index(drop=True)
    df.index = pd.DatetimeIndex(df['date'])
    for column in columns:
        df[column] = df.groupby(['Symbol','Name'])[column].ffill()
    df = df.reset_index(drop=True)
    return df

In [63]:
df = ffill_columns_values(df, ffill_columns)

In [64]:
df[df['Symbol'] == 'RAY']['tvl_solana']

31229    4.111275e+08
31230    3.816011e+08
31231    4.335738e+08
31232    4.895894e+08
31233    5.891269e+08
             ...     
31406    8.976373e+08
31407    9.303872e+08
31408    9.367030e+08
31409    9.152664e+08
31410    9.185706e+08
Name: tvl_solana, Length: 182, dtype: float64

In [65]:
df['Marketcap CMC'].unique().tolist()

[281403381.81,
 284506849.02,
 821962134.0,
 828663531.33,
 676569309.22,
 806459408.96,
 875552444.19,
 978210520.89,
 846523200.01,
 1349410494.9,
 1236237399.28,
 1326369228.61,
 1301525321.06,
 1436325183.17,
 1357918821.56,
 1368904971.36,
 1455107196.22,
 1357386684.68,
 1527744618.96,
 1526372332.15,
 1576810308.97,
 1603283280.42,
 1470760690.79,
 1320194829.74,
 1486812198.61,
 1459862741.58,
 1515912199.05,
 1407782680.44,
 1305165230.06,
 1335359499.18,
 1426103230.78,
 1189971148.27,
 1044787043.28,
 1033591637.9,
 1150168440.49,
 1167935695.33,
 1282528321.1,
 1333140096.63,
 1240715960.41,
 1370012484.51,
 1608502302.0,
 1628279213.42,
 1723917974.12,
 1686160136.26,
 1798412540.05,
 1665099150.59,
 1669923506.9,
 1706492552.63,
 1644555961.95,
 1414980043.37,
 1302900477.12,
 1480387579.16,
 1381437014.55,
 1429017529.51,
 1343857183.96,
 1533742937.73,
 1526221046.38,
 1453301427.37,
 1421048790.96,
 1221225061.02,
 1115487775.25,
 1083525532.88,
 1060501709.01,
 100606

In [66]:
df.tvl_defistation.unique()

array([      nan, 74614746., 68067114., ...,   638401.,   622841.,
         651463.])

In [67]:
required_tags

['Binance Smart Chain Ecosystem',
 'Polkadot Ecosystem',
 'Cosmos Ecosystem',
 'Solana Ecosystem',
 'Avalanche Ecosystem',
 'HECO Chain Ecosystem',
 'Layer 1',
 'Decentralized Finance (DeFi)',
 'Yield Farming',
 'Decentralized Exchange Token (DEX)',
 'Governance',
 'DaoMaker Ecosystem',
 'Gaming',
 'Stablecoins',
 'Asset Manager',
 'Centralized Exchange Token (CEX)',
 'Eth 2.0 Staking',
 'Yield Aggregator',
 'Lending/Borrowing',
 'Options',
 'Launchpad']

### Adding cmc tags as column

In [68]:
def append_tags_in_columns(df):
    required_tags.sort()
    tag_df = pd.DataFrame(df['Tags'].copy())

    required_tags_map = {}
    i = 0
    for tag in required_tags:
        required_tags_map[tag] = i
        i = i + 1
        
    all_symbol_tags = []
    for index, row in tag_df.iterrows():
        tag = [''] * (len(required_tags))
        tags = row['Tags']
        tags = tags.split(',') if tags else []
        for t in tags:
            index = required_tags_map[t] if t in required_tags_map else -1
            if index > -1:
                tag[index] = 1
        all_symbol_tags.append(tag)

    tag_columns = []
    tag_columns.extend(required_tags)
    tag_column_df = pd.DataFrame(all_symbol_tags, columns = tag_columns)
    df = df.join(tag_column_df)
    del df['Tags']
    df.reset_index(inplace=True, drop=True)
    return df

In [69]:
df = append_tags_in_columns(df)

Filter symbols.

In [70]:
exceptional_coins = ['BTC', 'ETH', 'BNB', 'SOL']
df = df[(df['Symbol'].isin(exceptional_coins)) | (df['Decentralized Finance (DeFi)'] == 1) | (df['required'])]
del df['required']
# df = df[(df['defi'] == 1)]



In [71]:
len(df.Name.unique())

292

In [72]:
df[df['Symbol'] == 'JOE']

,date,Symbol,Name,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Coingecko,...,HECO Chain Ecosystem,Launchpad,Layer 1,Lending/Borrowing,Options,Polkadot Ecosystem,Solana Ecosystem,Stablecoins,Yield Aggregator,Yield Farming
40587,2021-08-09,JOE,JOE,0.034519,0.000000e+00,132490.16,0.0,0.0,0.000000e+00,2.641678e+07,...,,,,,,,,,,
40588,2021-08-10,JOE,JOE,0.034519,0.000000e+00,132490.16,0.0,0.0,0.000000e+00,2.505540e+07,...,,,,,,,,,,
40589,2021-08-11,JOE,JOE,0.032930,0.000000e+00,108022.29,0.0,0.0,0.000000e+00,2.346859e+07,...,,,,,,,,,,
40590,2021-08-12,JOE,JOE,0.028150,0.000000e+00,337297.17,0.0,0.0,0.000000e+00,2.313089e+07,...,,,,,,,,,,
40591,2021-08-13,JOE,JOE,0.028902,0.000000e+00,100933.98,0.0,0.0,0.000000e+00,2.305233e+07,...,,,,,,,,,,
40592,2021-08-14,JOE,JOE,0.029080,0.000000e+00,87467.90,0.0,0.0,0.000000e+00,2.291475e+07,...,,,,,,,,,,
40593,2021-08-15,JOE,JOE,0.033736,0.000000e+00,188103.49,0.0,0.0,0.000000e+00,2.076003e+07,...,,,,,,,,,,
40594,2021-08-16,JOE,JOE,0.034692,0.000000e+00,99282.35,0.0,0.0,0.000000e+00,2.434043e+07,...,,,,,,,,,,
40595,2021-08-17,JOE,JOE,0.032698,0.000000e+00,132087.83,0.0,0.0,0.000000e+00,2.433395e+07,...,,,,,,,,,,
40596,2021-08-18,JOE,JOE,0.036597,0.000000e+00,370884.36,0.0,0.0,0.000000e+00,2.673547e+07,...,,,,,,,,,,


In [73]:
original_df

,date,Symbol,Name,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Coingecko,...,Revenue Supply Side TT,Revenue Protocol TT,Price/Sale,GMV TT,tvl_defistation,users_defistation,tvl_solana,id,Tags,required
0,2020-12-22,ZRX,0x,0.375319,2.814034e+08,4.782253e+07,7.497721e+08,NaN,1.000000e+09,NaN,...,0.0,5081.518355,327.001984,5.320025e+07,NaN,NaN,NaN,0x,"Decentralized Exchange Token (DEX),Polygon Eco...",True
1,2020-12-21,ZRX,0x,0.379458,2.845068e+08,7.704968e+07,7.497721e+08,NaN,1.000000e+09,NaN,...,0.0,4324.639219,351.334865,6.543229e+07,NaN,NaN,NaN,0x,"Decentralized Exchange Token (DEX),Polygon Eco...",True
2,2021-05-25,ZRX,0x,0.976579,8.219621e+08,1.149969e+08,8.416750e+08,1.000000e+09,1.000000e+09,NaN,...,0.0,20635.444305,53.273242,4.951713e+08,NaN,NaN,NaN,0x,"Decentralized Exchange Token (DEX),Polygon Eco...",True
3,2021-05-24,ZRX,0x,0.984541,8.286635e+08,1.398001e+08,8.416750e+08,1.000000e+09,1.000000e+09,NaN,...,0.0,29366.033133,53.631765,7.224367e+08,NaN,NaN,NaN,0x,"Decentralized Exchange Token (DEX),Polygon Eco...",True
4,2021-05-23,ZRX,0x,0.803837,6.765693e+08,1.512389e+08,8.416750e+08,1.000000e+09,1.000000e+09,NaN,...,0.0,55391.462027,64.133494,1.048609e+09,NaN,NaN,NaN,0x,"Decentralized Exchange Token (DEX),Polygon Eco...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40606,2021-08-24,BURGER,Burger Swap,5.830000,8.937555e+07,1.389763e+07,0.000000e+00,2.100000e+07,1.533028e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,burger-swap,"Decentralized Exchange Token (DEX),Automated M...",True
40607,2021-08-23,SBR,Saber,0.276757,2.758335e+07,3.251160e+05,9.966620e+07,1.000000e+10,1.001302e+08,629898012.0,...,NaN,NaN,NaN,NaN,NaN,NaN,6.301253e+08,saber,"Solana Ecosystem,",True
40608,2021-08-24,SBR,Saber,0.306437,3.075822e+07,1.986650e+05,9.966620e+07,1.000000e+10,1.001302e+08,639398054.0,...,NaN,NaN,NaN,NaN,NaN,NaN,6.382561e+08,saber,"Solana Ecosystem,",True
40609,2021-08-23,SCREAM,Scream,14.330000,2.866000e+07,1.279520e+05,0.000000e+00,2.000000e+06,2.000000e+06,25369377.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,scream,",",True


In [77]:
df[df['Symbol'] == 'JOE']

,date,Symbol,Name,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Coingecko,...,Yield Farming,TVL_max,GMV,Defi Volume DeBank,Defi Volume_max,fully_diluted_mkt_cap,total_revenue,total_supply_revenue,total_protocol_revenue,users_life_max
40587,2021-08-09,JOE,JOE,0.034519,0.000000e+00,132490.16,0.0,0.0,0.000000e+00,2.641678e+07,...,,2.641678e+07,1418956.68,NaN,1418956.68,0.0,NaN,0.0,0.0,NaN
40588,2021-08-10,JOE,JOE,0.034519,0.000000e+00,132490.16,0.0,0.0,0.000000e+00,2.505540e+07,...,,2.505540e+07,1326369.61,NaN,1326369.61,0.0,NaN,0.0,0.0,NaN
40589,2021-08-11,JOE,JOE,0.032930,0.000000e+00,108022.29,0.0,0.0,0.000000e+00,2.346859e+07,...,,2.346859e+07,971290.74,NaN,971290.74,0.0,NaN,0.0,0.0,NaN
40590,2021-08-12,JOE,JOE,0.028150,0.000000e+00,337297.17,0.0,0.0,0.000000e+00,2.313089e+07,...,,2.313089e+07,737123.18,NaN,737123.18,0.0,NaN,0.0,0.0,NaN
40591,2021-08-13,JOE,JOE,0.028902,0.000000e+00,100933.98,0.0,0.0,0.000000e+00,2.305233e+07,...,,2.305233e+07,819679.08,NaN,819679.08,0.0,NaN,0.0,0.0,NaN
40592,2021-08-14,JOE,JOE,0.029080,0.000000e+00,87467.90,0.0,0.0,0.000000e+00,2.291475e+07,...,,2.291475e+07,599596.18,NaN,599596.18,0.0,NaN,0.0,0.0,NaN
40593,2021-08-15,JOE,JOE,0.033736,0.000000e+00,188103.49,0.0,0.0,0.000000e+00,2.076003e+07,...,,2.076003e+07,1533653.16,NaN,1533653.16,0.0,NaN,0.0,0.0,NaN
40594,2021-08-16,JOE,JOE,0.034692,0.000000e+00,99282.35,0.0,0.0,0.000000e+00,2.434043e+07,...,,2.434043e+07,3723993.98,NaN,3723993.98,0.0,NaN,0.0,0.0,NaN
40595,2021-08-17,JOE,JOE,0.032698,0.000000e+00,132087.83,0.0,0.0,0.000000e+00,2.433395e+07,...,,2.433395e+07,6427245.27,NaN,6427245.27,0.0,NaN,0.0,0.0,NaN
40596,2021-08-18,JOE,JOE,0.036597,0.000000e+00,370884.36,0.0,0.0,0.000000e+00,2.673547e+07,...,,2.673547e+07,10880539.18,NaN,10880539.18,0.0,NaN,0.0,0.0,NaN


### adding columns like TVL_max, Defi Volume_max, users_life_max and etc.

In [75]:
def add_other_columns(df):
    # df['TVL_max'] = df[['TVL Debank', 'TVL Defipulse']].max(axis=1)
    # df['TVL_max'] = df[['TVL Debank', 'TVL Defipulse', 'tvl_defistation']].max(axis=1)
    df['TVL_max'] = df[['TVL Debank', 'tvl_defistation', 'tvl_solana', 'TVL Coingecko']].max(axis=1)
    df['GMV'] = df['GMV TT']
    df['Defi Volume DeBank'] = (df['Trading Volume Debank'].fillna(0) + df['Borrowing Volume Debank'].fillna(0)).replace({'0': np.nan, 0: np.nan})
    # add defi volume max as max of gmv from tokenterminal and defi volume debank
    df['Defi Volume_max'] = df[['GMV', 'Defi Volume DeBank']].max(axis=1)
    # add defi volume max as max of gmv from tokenterminal and defi volume debank
    df['Defi Volume_max'] = df[['GMV', 'Defi Volume DeBank']].max(axis=1)
    df['max_test'] = df[['MAX Supply CMC', 'Total Supply CMC']].max(axis=1)
    df['fully_diluted_mkt_cap'] = df.max_test * df.Price
    df["total_revenue"] = df['Revenue TT']
    df["total_supply_revenue"] = df['Revenue Supply Side TT']
    df["total_protocol_revenue"] = df['Revenue Protocol TT']
    df['users_life_max'] = df[['users_defistation', 'Users life-to-date duneanlytics']].max(axis=1)
    del df['max_test']
    df = df.sort_values('date')
    return df

In [76]:
df = add_other_columns(df)

In [ ]:
def select_required_metrics_coins_only(df):

    def fillter_coins_basis_metrics(df, column, symbol):
        tmp_df = df[df['Symbol'] == symbol][column].isnull()
        return tmp_df.sum() == tmp_df.count()

    to_delete_symbols = []
    required_metrics = ['TVL_max', 'Defi Volume_max', 'total_revenue']
    symbols = df.Symbol.unique().tolist()
    for symbol in symbols:
        to_delete = True
        for metric in required_metrics:
            to_delete = fillter_coins_basis_metrics(df, metric, symbol)
            if not to_delete:
                break
        to_delete_symbols.append([symbol, to_delete])
    to_delete_symbols_df = pd.DataFrame(to_delete_symbols, columns=['Symbol', 'to_delete'])
    to_delete_symbols_df = to_delete_symbols_df[to_delete_symbols_df['to_delete']]
    to_delete_symbols = to_delete_symbols_df['Symbol'].values.tolist()
    df2 = df.drop(df[df['Symbol'].isin(to_delete_symbols)].index)
    df2 = df2.reset_index(drop=True)
    return df2

In [ ]:
df = select_required_metrics_coins_only(df)
df

In [ ]:
cmc_symbol_name_df = get_all_other_datasource_symbols()
cmc_symbol_name_df['Symbol'] = cmc_symbol_name_df['symbol']
cmc_symbol_name_df['Name'] = cmc_symbol_name_df['name_in_coingecko']

In [ ]:
cmc_symbol_name_df

In [ ]:
df[df['Marketcap CMC'] > 10000000].reset_index(drop=True)

### Adding all ratios

In [ ]:
# add n day users
def calculate_nDay_users(df, days):
    column_name = f"total_users_{days}d"
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df['24hr_Debank_users_max'].rolling(f"{days}d").sum()
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]

In [ ]:
def calculate_nDay_revenue(df, days):
    column_name = f"total_revenue_{days}d"
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df['total_revenue'].rolling(f"{days}d").sum()
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]
    

In [ ]:
# function for defi volume sums
# substitute for volume cmc sum
# changing defi volume debank to Defi Volume_max
def calculate_nDay_defi_volume(df, days):
    column_name = f"total_defi_volume_{days}d"
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df['Defi Volume_max'].rolling(f"{days}d").sum()
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]
            

In [ ]:
def calculate_nDay_change(df, days, column, column_name):
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df[column].pct_change(days)
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]
    
def calculate_nDay_std(df, days, column, column_name, div_by_column='Price'):
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df[column].rolling(days).std() / temp_df[div_by_column]
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]
    

In [ ]:
def add_ratios_and_pct_change(df):
    calculate_nDay_revenue(df, 7)
    calculate_nDay_revenue(df, 30)
    #calculate_nDay_trading_volume(df, 30)
    calculate_nDay_defi_volume(df, 30)

    # add n day users
    calculate_nDay_users(df, 30)

    # add 1, 3, 7, 30 day defi volume pct change here
    # added column calculations
    #
    calculate_nDay_change(df, 1, 'Defi Volume_max', 'Defi_volume_pct_chg_1d')
    calculate_nDay_change(df, 3, 'Defi Volume_max', 'Defi_volume_pct_chg_3d')
    calculate_nDay_change(df, 7, 'Defi Volume_max', 'Defi_volume_pct_chg_7d')
    calculate_nDay_change(df, 30, 'Defi Volume_max', 'Defi_volume_pct_chg_30d')
    #
    # flip such that mc in numerator 
    #df['VOL : MC'] = df['Volume CMC'] / df['Marketcap CMC']
    df['MC : VOL'] = df['Marketcap CMC'] / df['Volume CMC'] 
    # added column calculations
    calculate_nDay_change(df, 1, 'MC : VOL', 'MC_VOL_pct_chg_1d')
    calculate_nDay_change(df, 3, 'MC : VOL', 'MC_VOL_pct_chg_3d')
    calculate_nDay_change(df, 7, 'MC : VOL', 'MC_VOL_pct_chg_7d')
    calculate_nDay_change(df, 30, 'MC : VOL', 'MC_VOL_pct_chg_30d')
    #

    # flip such that mc in numerator
    #df['VOL : DMC'] = df['Volume CMC'] / df['fully_diluted_mkt_cap']
    df['DMC : VOL'] = df['fully_diluted_mkt_cap'] / df['Volume CMC']
    # added column calculations
    calculate_nDay_change(df, 1, 'DMC : VOL', 'DMC_VOL_pct_chg_1d')
    calculate_nDay_change(df, 3, 'DMC : VOL', 'DMC_VOL_pct_chg_3d')
    calculate_nDay_change(df, 7, 'DMC : VOL', 'DMC_VOL_pct_chg_7d')
    calculate_nDay_change(df, 30, 'DMC : VOL', 'DMC_VOL_pct_chg_30d')
    #



    df['MC : TVL'] = df['Marketcap CMC'] / df['TVL_max']
    # added column calculations
    calculate_nDay_change(df, 1, 'MC : TVL', 'MC_TVL_pct_chg_1d')
    calculate_nDay_change(df, 3, 'MC : TVL', 'MC_TVL_pct_chg_3d')
    calculate_nDay_change(df, 7, 'MC : TVL', 'MC_TVL_pct_chg_7d')
    calculate_nDay_change(df, 30, 'MC : TVL', 'MC_TVL_pct_chg_30d')
    #


    df['DMC : TVL'] = df['fully_diluted_mkt_cap'] / df['TVL_max']
    # added column calculations
    calculate_nDay_change(df, 1, 'DMC : TVL', 'DMC_TVL_pct_chg_1d')
    calculate_nDay_change(df, 3, 'DMC : TVL', 'DMC_TVL_pct_chg_3d')
    calculate_nDay_change(df, 7, 'DMC : TVL', 'DMC_TVL_pct_chg_7d')
    calculate_nDay_change(df, 30, 'DMC : TVL', 'DMC_TVL_pct_chg_30d')
    #


    df['MC : users (1d)'] = df['Marketcap CMC'] / df['24hr_Debank_users_max']
    # added column calculations
    calculate_nDay_change(df, 1, 'MC : users (1d)', 'MC_users_pct_chg_1d')
    calculate_nDay_change(df, 3, 'MC : users (1d)', 'MC_users_pct_chg_3d')
    calculate_nDay_change(df, 7, 'MC : users (1d)', 'MC_users_pct_chg_7d')
    calculate_nDay_change(df, 30, 'MC : users (1d)', 'MC_users_pct_chg_30d')
    #




    df['DMC : users (1d)'] = df['fully_diluted_mkt_cap'] / df['24hr_Debank_users_max']
    # added column calculations
    calculate_nDay_change(df, 1, 'DMC : users (1d)', 'DMC_users_pct_chg_1d')
    calculate_nDay_change(df, 3, 'DMC : users (1d)', 'DMC_users_pct_chg_3d')
    calculate_nDay_change(df, 7, 'DMC : users (1d)', 'DMC_users_pct_chg_7d')
    calculate_nDay_change(df, 30, 'DMC : users (1d)', 'DMC_users_pct_chg_30d')
    #



    df['MC : transactions (1d)'] = df['Marketcap CMC'] / df['Transactions Count Debank']
    # added column calculations
    calculate_nDay_change(df, 1, 'MC : transactions (1d)', 'MC_transactions_pct_chg_1d')
    calculate_nDay_change(df, 3, 'MC : transactions (1d)', 'MC_transactions_pct_chg_3d')
    calculate_nDay_change(df, 7, 'MC : transactions (1d)', 'MC_transactions_pct_chg_7d')
    calculate_nDay_change(df, 30, 'MC : transactions (1d)', 'MC_transactions_pct_chg_30d')
    #

    df['DMC : transactions (1d)'] = df['fully_diluted_mkt_cap'] / df['Transactions Count Debank']
    # added column calculations
    calculate_nDay_change(df, 1, 'DMC : transactions (1d)', 'DMC_transactions_pct_chg_1d')
    calculate_nDay_change(df, 3, 'DMC : transactions (1d)', 'DMC_transactions_pct_chg_3d')
    calculate_nDay_change(df, 7, 'DMC : transactions (1d)', 'DMC_transactions_pct_chg_7d')
    calculate_nDay_change(df, 30, 'DMC : transactions (1d)', 'DMC_transactions_pct_chg_30d')
    #

    #df['MC : DEX Vol (1d)'] = df['Marketcap CMC'] / df['Trading Volume Debank']
    # add
    df['MC : Defi Vol (1d)'] = df['Marketcap CMC'] / df['Defi Volume_max']
    # added column calculations
    calculate_nDay_change(df, 1, 'MC : Defi Vol (1d)', 'MC_defi_volume_pct_chg_1d')
    calculate_nDay_change(df, 3, 'MC : Defi Vol (1d)', 'MC_defi_volume_pct_chg_3d')
    calculate_nDay_change(df, 7, 'MC : Defi Vol (1d)', 'MC_defi_volume_pct_chg_7d')
    calculate_nDay_change(df, 30, 'MC : Defi Vol (1d)', 'MC_defi_volume_pct_chg_30d')
    #


    df['MC : Defi Vol (30d)'] = df['Marketcap CMC'] / df['total_defi_volume_30d']
    #df['DMC : DEX Vol (1d)'] = df['fully_diluted_mkt_cap'] / df['Trading Volume Debank']
    #df['DMC : DEX Vol (30d)'] = df['fully_diluted_mkt_cap'] / df['total_volume_30d']



    df['MC : revenue'] = df['Marketcap CMC'] / df['total_revenue']
    # added column calculations
    calculate_nDay_change(df, 1, 'MC : revenue', 'MC_revenue_pct_chg_1d')
    calculate_nDay_change(df, 3, 'MC : revenue', 'MC_revenue_pct_chg_3d')
    calculate_nDay_change(df, 7, 'MC : revenue', 'MC_revenue_pct_chg_7d')
    calculate_nDay_change(df, 30, 'MC : revenue', 'MC_revenue_pct_chg_30d')
    #

    df['DMC : revenue'] = df['fully_diluted_mkt_cap'] / df['total_revenue']
    # added column calculations
    calculate_nDay_change(df, 1, 'DMC : revenue', 'DMC_revenue_pct_chg_1d')
    calculate_nDay_change(df, 3, 'DMC : revenue', 'DMC_revenue_pct_chg_3d')
    calculate_nDay_change(df, 7, 'DMC : revenue', 'DMC_revenue_pct_chg_7d')
    calculate_nDay_change(df, 30, 'DMC : revenue', 'DMC_revenue_pct_chg_30d')
    #


    df['MC : total_users_30d'] = df['Marketcap CMC'] / df['total_users_30d']
    # added column calculations
    calculate_nDay_change(df, 1, 'MC : total_users_30d', 'MC_total_users_30d_pct_chg_1d')
    calculate_nDay_change(df, 3, 'MC : total_users_30d', 'MC_total_users_30d_pct_chg_3d')
    calculate_nDay_change(df, 7, 'MC : total_users_30d', 'MC_total_users_30d_pct_chg_7d')
    calculate_nDay_change(df, 30, 'MC : total_users_30d', 'MC_total_users_30d_pct_chg_30d')
    #

    df['DMC : total_users_30d'] = df['fully_diluted_mkt_cap'] / df['total_users_30d']
    # added column calculations
    calculate_nDay_change(df, 1, 'DMC : total_users_30d', 'DMC_total_users_30d_pct_chg_1d')
    calculate_nDay_change(df, 3, 'DMC : total_users_30d', 'DMC_total_users_30d_pct_chg_3d')
    calculate_nDay_change(df, 7, 'DMC : total_users_30d', 'DMC_total_users_30d_pct_chg_7d')
    calculate_nDay_change(df, 30, 'DMC : total_users_30d', 'DMC_total_users_30d_pct_chg_30d')
    #

    df['MC : Users_life'] = df['Marketcap CMC'] / df['users_life_max']
    # added column calculations
    calculate_nDay_change(df, 1, 'MC : Users_life', 'MC_Users_life_pct_chg_1d')
    calculate_nDay_change(df, 3, 'MC : Users_life', 'MC_Users_life_pct_chg_3d')
    calculate_nDay_change(df, 7, 'MC : Users_life', 'MC_Users_life_pct_chg_7d')
    calculate_nDay_change(df, 30, 'MC : Users_life', 'MC_Users_life_pct_chg_30d')
    #
    df['DMC : Users_life'] = df['fully_diluted_mkt_cap'] / df['users_life_max']
    # added column calculations
    calculate_nDay_change(df, 1, 'DMC : Users_life', 'DMC_Users_life_pct_chg_1d')
    calculate_nDay_change(df, 3, 'DMC : Users_life', 'DMC_Users_life_pct_chg_3d')
    calculate_nDay_change(df, 7, 'DMC : Users_life', 'DMC_Users_life_pct_chg_7d')
    calculate_nDay_change(df, 30, 'DMC : Users_life', 'DMC_Users_life_pct_chg_30d')
    #
    calculate_nDay_change(df, 1, 'Price', 'Price_pct_chg_1d')
    calculate_nDay_change(df, 3, 'Price', 'Price_pct_chg_3d')
    calculate_nDay_change(df, 7, 'Price', 'Price_pct_chg_7d')
    calculate_nDay_change(df, 30, 'Price', 'Price_pct_chg_30d')
    calculate_nDay_std(df, 7, 'Price', 'Price_7day_volatility_pct', div_by_column='Price')
    calculate_nDay_std(df, 30, 'Price', 'Price_30day_volatility_pct', div_by_column='Price')
    calculate_nDay_std(df, 90, 'Price', 'Price_90day_volatility_pct', div_by_column='Price')
    #
    calculate_nDay_change(df, 1, 'Marketcap CMC', 'MC_pct_chg_1d')
    calculate_nDay_change(df, 3, 'Marketcap CMC', 'MC_pct_chg_3d')
    calculate_nDay_change(df, 7, 'Marketcap CMC', 'MC_pct_chg_7d')
    calculate_nDay_change(df, 30, 'Marketcap CMC', 'MC_pct_chg_30d')
    calculate_nDay_std(df, 30, 'Marketcap CMC', 'MC_30day_volatility_pct', div_by_column='Marketcap CMC')
    calculate_nDay_std(df, 90, 'Marketcap CMC', 'MC_90day_volatility_pct', div_by_column='Marketcap CMC')
    #
    calculate_nDay_change(df, 1, 'fully_diluted_mkt_cap', 'DMC_pct_chg_1d')
    calculate_nDay_change(df, 3, 'fully_diluted_mkt_cap', 'DMC_pct_chg_3d')
    calculate_nDay_change(df, 7, 'fully_diluted_mkt_cap', 'DMC_pct_chg_7d')
    calculate_nDay_change(df, 30, 'fully_diluted_mkt_cap', 'DMC_pct_chg_30d')
    calculate_nDay_std(df, 30, 'fully_diluted_mkt_cap', 'DMC_30day_volatility_pct', div_by_column='fully_diluted_mkt_cap')
    calculate_nDay_std(df, 90, 'fully_diluted_mkt_cap', 'DMC_90day_volatility_pct', div_by_column='fully_diluted_mkt_cap')
    #
    calculate_nDay_change(df, 1, 'Circulating supply CMC', 'CS_pct_chg_1d')
    calculate_nDay_change(df, 3, 'Circulating supply CMC', 'CS_pct_chg_d')
    calculate_nDay_change(df, 7, 'Circulating supply CMC', 'CS_pct_chg_7d')
    calculate_nDay_change(df, 30, 'Circulating supply CMC', 'CS_pct_chg_30d')
    calculate_nDay_std(df, 30, 'Circulating supply CMC', 'CS_30day_volatility_pct', div_by_column='Circulating supply CMC')
    calculate_nDay_std(df, 90, 'Circulating supply CMC', 'CS_90day_volatility_pct', div_by_column='Circulating supply CMC')
    #
    calculate_nDay_change(df, 1, 'Volume CMC', 'Volume_pct_chg_1d')
    calculate_nDay_change(df, 3, 'Volume CMC', 'Volume_pct_chg_3d')
    calculate_nDay_change(df, 7, 'Volume CMC', 'Volume_pct_chg_7d')
    calculate_nDay_change(df, 30, 'Volume CMC', 'Volume_pct_chg_30d')
    calculate_nDay_std(df, 30, 'Volume CMC', 'Volume_30day_volatility_pct', div_by_column='Volume CMC')
    calculate_nDay_std(df, 90, 'Volume CMC', 'Volume_90day_volatility_pct', div_by_column='Volume CMC')
    #
    calculate_nDay_change(df, 1, 'TVL_max', 'TVL_pct_chg_1d')
    calculate_nDay_change(df, 3, 'TVL_max', 'TVL_pct_chg_3d')
    calculate_nDay_change(df, 7, 'TVL_max', 'TVL_pct_chg_7d')
    calculate_nDay_change(df, 30, 'TVL_max', 'TVL_pct_chg_30d')
    calculate_nDay_std(df, 30, 'TVL_max', 'TVL_30day_volatility_pct', div_by_column='TVL_max')
    calculate_nDay_std(df, 90, 'TVL_max', 'TVL_90day_volatility_pct', div_by_column='TVL_max')
    #
    calculate_nDay_change(df, 1, 'total_revenue', 'Rev_pct_chg_1d')
    calculate_nDay_change(df, 3, 'total_revenue', 'Rev_pct_chg_3d')
    calculate_nDay_change(df, 7, 'total_revenue', 'Rev_pct_chg_7d')
    calculate_nDay_change(df, 30, 'total_revenue', 'Rev_pct_chg_30d')
    
    calculate_nDay_change(df, 1, 'Circulating supply CMC', 'emission_rate')
    
    calculate_nDay_std(df, 30, 'total_revenue', 'Rev_30day_volatility_pct', div_by_column='total_revenue')
    calculate_nDay_std(df, 90, 'total_revenue', 'Rev_90day_volatility_pct', div_by_column='total_revenue')
    df['Annual revenue (estimate)'] = df['total_revenue_30d'] * (1 + df['Rev_pct_chg_30d'])*12
    return df


In [ ]:
# df = add_ratios_and_pct_change(df)

In [ ]:
df

In [ ]:
def arrange_columns(df):
    columns = df.columns.tolist()
    for tag in required_tags:
        columns.remove(tag)
    columns.extend(required_tags)
    return df[columns]

In [ ]:
df = arrange_columns(df)

### Generating neg div score

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
import numpy as np
import pymysql
import traceback2


def read_csv_as_df():
    df = pd.read_csv('/tmp/coins_data.csv')
    df.index = pd.DatetimeIndex(df['time'])
    del df['time']
    df = df[df.index >= datetime(2020, 12, 26)]
    return df

def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()


def insert_defi_neg_div_score(df, date):
    connection = __connect__()
    cursor = get_cursor(connection)
    dt = date.timestamp()
    for i, row in df.iterrows():
        try:
            sys_name = row["Symbol"]
            name = sys_name.split("###")[1]
            symbol = sys_name.split("###")[0]
            print(dt, date, symbol, name, row['C_Price'], row['3_Day_LB'], row['5_Day_LB'])
            sql = "INSERT INTO `defi_neg_div_score` (`timestamp_dt`, `symbol`, `name`, `price`, `3_Day_LB`, `5_Day_LB`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s)"
            cursor.execute(sql, (dt, symbol, name, row['C_Price'], row['3_Day_LB'], row['5_Day_LB']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()

def clean_data(df):
    index = df.index.values
#     print(index)
    if 'USDT' in index:
        del df['USDT']
    if 'USDC' in index:
        del df['USDC']
    if 'DAI' in index:
        del df['DAI']
    if 'BUSD' in index:
        del df['BUSD']
    if 'HUSD' in index:
        del df['HUSD']
    if 'TUSD' in index:
        del df['TUSD']
    if 'BTC' in index:
        del df['BTC']
    data_values_df = pd.DataFrame(df.isna().sum(), columns=['nan_count'])
    tmp_df = pd.DataFrame(df.count(), columns=['values_count'])
    data_values_df['values_count'] = tmp_df['values_count']
    data_values_df['rows_count'] = data_values_df['values_count'] + data_values_df['nan_count']
#     print(data_values_df)
    delete_coin_df = data_values_df[data_values_df['nan_count'] > (data_values_df['values_count'] * .20)]
    delete_coin_df.sort_values(['nan_count'])
    symbols_del = delete_coin_df.index.tolist()
    for symbol in symbols_del:
        print(f'Deleteing Coin: {symbol}')
        del df[symbol]
    df.ffill(inplace=True)
    df = df.dropna(axis=1)
    df.index = df.index
    return df


def get_todays_portfolio_balance(trades):
    balance = 0
    for symbol in trades:
        balance = balance + trades[symbol][3]
    return balance


def make_entry_symbol(df, trading_date, symbol, last_trade, fund):
    last_trade = last_trade.copy()
    todays_close = get_close_price_on(df, symbol, trading_date)
    if not len(last_trade):
        last_trade = [trading_date, todays_close, fund / todays_close, fund, 0]
        return last_trade
    last_trade[0] = trading_date
    last_trade[1] = todays_close
    last_trade[3] = last_trade[2] * todays_close
    last_trade[4] = last_trade[4] + 1
#     print(last_trade)
    return last_trade


def make_exit_symbol(df, trading_date, symbol, last_trade):
    quantity = last_trade[2]
    todays_close = get_close_price_on(df, symbol, trading_date)
    return quantity * todays_close


def get_not_in_list(new_list, old_list):
    not_in_new = []
    for symbol in old_list:
        if symbol not in new_list:
            not_in_new.append(symbol)
    return not_in_new


def get_commons(symbols1, symbols2):
    symbols_list = []
    for symbol in symbols2:
        if symbol in symbols1:
            symbols_list.append(symbol)
    return symbols_list


def get_joins(symbols1, symbols2):
    symbols_list = symbols1.copy()
    for symbol in symbols2:
        if symbol not in symbols1:
            symbols_list.append(symbol)
    return symbols_list


def get_todays_top_n_coin(df, trade_date, top_n, order='tail'):
    data = df.loc[trade_date]
    data = pd.DataFrame(data)
    data = data.rename(columns={data.columns[0]: 'C'})
    todays_close_df = pd.DataFrame(data)
    if order == 'head':
        todays_close_df = todays_close_df.sort_values(['C']).head(top_n)
    else:
        todays_close_df = todays_close_df.sort_values(['C']).tail(top_n)
    return todays_close_df.index.values.tolist()


def get_next_trading_day_as_index(start_date, hour):
    return pd.to_datetime((start_date + timedelta(days=1)).replace(hour=hour, minute=0, second=0).strftime("%Y %m %d %H:%M:00"))


def get_close_price_on(df, symbol, trade_date):
    data = df.loc[trade_date]
    data = pd.DataFrame(data)
    data = data.rename(columns={data.columns[0]: 'C'})
    todays_close_df = pd.DataFrame(data)
    try:
        return todays_close_df.loc[symbol]['C']
    except Exception:
        return 0


def get_qualified_coins(trade_date, factors_df, factors_count):
    factors_symbols = []
    index = 0
    for factor_df in factors_df:
        s1 = get_todays_top_n_coin(factor_df, trade_date, factors_count[index])
        index = index + 1
        factors_symbols.append(s1)
    symbols = factors_symbols[0]
    for index in range(1, len(factors_symbols)):
        symbols = get_commons(symbols, factors_symbols[index])
    return symbols


def update_coins_performance(coins_performence, todays_trading_symbols):
    symbol_performance = [0, 0]
    for symbol in todays_trading_symbols:
        if symbol in coins_performence:
            symbol_performance = coins_performence[symbol]
        else:
            symbol_performance = [0, 0]
            coins_performence[symbol] = symbol_performance
        symbol_performance[0] = symbol_performance[0] + 1


def update_total_days_count_in_coin_performance(coins_performence, days_count):
    for symbol in coins_performence:
        coins_performence[symbol][1] = days_count


def plot_portfolio_performence(trade_date, df, factors_df, factors_count):
    trades = {}
    trades['c-f'] = [trade_date, 0, 0, 0]
    last_day_trading_symbols = []
    is_first_day_trading = True
    portfolio_performence = []
    end_time = df.index.values.max()
    end_time = (end_time - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    coins_performence = {}
    days_count = 0
    while trade_date.timestamp() < end_time:
        days_count = days_count + 1
        todays_trading_symbols = get_qualified_coins(trade_date, factors_df, factors_count)
        update_coins_performance(coins_performence, todays_trading_symbols)
        if is_first_day_trading:
            is_first_day_trading = False
            if not len(todays_trading_symbols):
                trades['c-f'][3] = 100000
                continue
            fund = 100000 / len(todays_trading_symbols)
            for symbol in todays_trading_symbols:
                trade = make_entry_symbol(df, trade_date, symbol, [], fund)
                trades[symbol] = trade
#             is_first_day_trading = False
            last_day_trading_symbols = todays_trading_symbols.copy()
            continue
        tmp_trades = trades

        trades = {}
        exit_fund = tmp_trades['c-f'][3]
        todays_exit_symbols = get_not_in_list(todays_trading_symbols, last_day_trading_symbols)
        todays_common_symbols = get_commons(todays_trading_symbols, last_day_trading_symbols)
        todays_new_entry_symbols = get_not_in_list(todays_common_symbols, todays_trading_symbols)

        new_symbol = []
        for symbol_c in todays_common_symbols:
            trade = tmp_trades[symbol_c]
            if trade[4] > 5:
                exit_fund = exit_fund + make_exit_symbol(df, trade_date, symbol_c, trade)
                if symbol_c in todays_trading_symbols:
                    todays_new_entry_symbols.append(symbol_c)
                continue
            trade = make_entry_symbol(df, trade_date, symbol_c, trade, 0)
            trades[symbol_c] = trade

        for exit_symbol in todays_exit_symbols:
            exit_trade = tmp_trades[exit_symbol]
            fund = make_exit_symbol(df, trade_date, exit_symbol, exit_trade)
            exit_fund = exit_fund + fund
        if len(todays_new_entry_symbols):
            new_entry_fund = exit_fund / len(todays_new_entry_symbols)
            exit_fund = 0
            for entry_symbol in todays_new_entry_symbols:
                trade = make_entry_symbol(df, trade_date, entry_symbol, [], new_entry_fund)
                trades[entry_symbol] = trade
                new_symbol.append(entry_symbol)
        tmp_trades['c-f'][3] = exit_fund
        tmp_trades['c-f'][0] = trade_date
        trades['c-f'] = tmp_trades['c-f']
#         if tmp_trades['c-f'][3] > 0:
#             print(f"Uninvested amount as on {tmp_trades['c-f'][0]} is {tmp_trades['c-f'][3]}")
        portfolio_performence.append([get_todays_portfolio_balance(trades), trade_date])
        trade_date = get_next_trading_day_as_index(trade_date, trade_date.hour)
        last_day_trading_symbols = get_joins(todays_common_symbols, new_symbol)
    update_total_days_count_in_coin_performance(coins_performence, days_count)
    print(f'Last Trading Day: {trade_date}. Currently Coins in Portfolio: \n {todays_trading_symbols}')
    return [portfolio_performence, todays_trading_symbols, coins_performence]


def plot_portfolio_for_config(close_df, factors_pct, factors_df):
    total_symbols_count = len(close_df.columns.values.tolist())
    factors_count = []
    for factor_pct in factors_pct:
        factors_count.append(int(total_symbols_count / 100 * factor_pct))

    start_time = factors_df[0].index.values.min()
    start_time_secs = (start_time - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    start_time = datetime.utcfromtimestamp(start_time_secs)
    trade_date = start_time + timedelta(days=0)
    trade_date = get_next_trading_day_as_index(trade_date, trade_date.hour)
    print(f'Trading Start Date: {trade_date}')
    portfolio = plot_portfolio_performence(trade_date, close_df, factors_df, factors_count)
    portfolio_performence = portfolio[0]
    performance_df = pd.DataFrame(portfolio_performence, columns=['balance', 'time'])
    performance_df.index = pd.DatetimeIndex(performance_df['time'])
    del performance_df['time']
    performance_df.plot(figsize=[18, 10])
#     plt.savefig(f'/tmp/portfolio_{"_".join(str(x) for x in factors_pct)}.png')

    print(f'Current Portfilio Balance: {portfolio_performence[-1][0]}')
    return portfolio[1:]


def get_rsq(x):
    X = x.values.reshape(-1, 1)
    Y = np.arange(1, len(X) + 1).reshape(-1, 1)
    linear_regressor = LinearRegression(fit_intercept=True)
    linear_regressor.fit(Y, X)
    return linear_regressor.score(Y, X)
#     return r2_score(Y, X)


def get_portfolio_version_coins(close_df, factors_df):
    # factors_df = [_7_days_pct_change_df, _days_sumed_pct_change_df, _6_hour_rsq_df, _pure_div_score]
    factors = [25, 10, 10, 99]
    coins_performance = plot_portfolio_for_config(close_df, factors, factors_df)
    coin_performance_df = pd.DataFrame.from_dict(coins_performance[1], orient='index', columns=['Occurrences', 'Total Trading Day'])
    coin_performance_df = coin_performance_df.sort_values(['Occurrences'], ascending=True)
#     coin_performance_df.to_csv(f'/tmp/portfolio_{"_".join(str(x) for x in factors)}_coin_performance.csv')
    s1 = coins_performance[0]
    factors = [25, 99, 50, 7.5]
    coins_performance = plot_portfolio_for_config(close_df, factors, factors_df)
    coin_performance_df = pd.DataFrame.from_dict(coins_performance[1], orient='index', columns=['Occurrences', 'Total Trading Day'])
    coin_performance_df = coin_performance_df.sort_values(['Occurrences'], ascending=True)
#     coin_performance_df.to_csv(f'/tmp/portfolio_{"_".join(str(x) for x in factors)}_coin_performance.csv')
    s2 = coins_performance[0]
    return get_joins(s1, s2)


def run_portfolio_version(close_df, factors_df, _6_hour_resampled_df, day):
    _7_days_pct_change_df = _6_hour_resampled_df.pct_change(periods=day * 4)
    factors_df[0] = _7_days_pct_change_df
    return get_portfolio_version_coins(close_df, factors_df)


def get_coins_price(close_df, symbols):
    df = close_df.iloc[[-1]]
    symbols_price = f"As On {df.index.values[0]}<br>"
    for symbol in symbols:
        symbols_price = f'{symbols_price} {symbol}: ${df.iloc[0][symbol]},'
    return symbols_price


def get_lookback_df(close_df, lookback_symbols):
    close_df = close_df.iloc[[-1]]
    symbols = []
    for lookback in lookback_symbols:
        symbols = get_joins(symbols, lookback_symbols[lookback])
    data = []
    for symbol in symbols:
        data.append([
            symbol,
            close_df.iloc[0][symbol],
            symbol in lookback_symbols[3],
            symbol in lookback_symbols[5]
        ])
    df = pd.DataFrame(data, columns=['Symbol', 'C_Price', '3_Day_LB', '5_Day_LB'])
    return df


def run_algo(close_df):
    _6_hour_resampled_df = close_df.resample('6H').last()
    _6_hour_pct_change = _6_hour_resampled_df.pct_change(periods=1)
    _days_sumed_pct_change_df = _6_hour_pct_change.rolling(window=4).sum()
    _pure_div_score = _6_hour_pct_change.applymap(lambda x: x if x < 0 else 0)
    _pure_div_score = _pure_div_score.rolling(window=4).sum()
    _6_hour_rsq_df = _6_hour_resampled_df.rolling(window=28).apply(get_rsq)
    factors_df = [None, _days_sumed_pct_change_df, _6_hour_rsq_df, _pure_div_score]
    # return run_portfolio_version(close_df, factors_df, _6_hour_resampled_df, 7)
    days = [3, 5]
    msg = ""
    lookback_symbols = {}
    for day in days:
        msg = f'{msg}<br> {day}-day Lookback. <br>Coins Symbol in Portfolio are as below:<br>'
        symbols = run_portfolio_version(close_df, factors_df, _6_hour_resampled_df, day)
        lookback_symbols[day] = symbols
        msg = f'{msg} {get_coins_price(close_df, symbols)}<br><br>'
    lookback_df = get_lookback_df(close_df, lookback_symbols)
    return lookback_df


def delete_unnessecary_columns(df):
    df = df.drop_duplicates(keep='first')
    return df


def convert_symbols_to_columns(df):
    df = df.pivot(columns='symbol', values='C')
    return df


def get_symbols_close(ids):
    connection = __connect__()
    s = ",".join([f'"{s}"' for s in ids])
    sql = f"select * from coingecko_coin_hist_data where symbol in ({s})"
    print(sql)
    df = pd.read_sql(sql, connection)
    connection.close()
    return df

def get_symbols_close_data_df(data_df):
    ids = data_df['Symbol'].unique().tolist()
    df = get_symbols_close(ids)
    df['sym-name']  = df['symbol'] +'###'+df['name']
    df['symbol'] = df['sym-name']
    df['C'] = df['price']
    df = df[['symbol', 'C', 'timestamp_dt'] ].copy()
    df['timestamp_dt'] = pd.to_datetime(df['timestamp_dt'])
    df.index = pd.DatetimeIndex(df['timestamp_dt'])
    df.index = df.index + timedelta(seconds=60)
    df = df[df.index >= datetime(2021, 1, 15)]
    df = delete_unnessecary_columns(df)
    df = convert_symbols_to_columns(df)
    return df


def run_backtest(close_df):
    df = clean_data(close_df)
    print(f'1st Trading Day: {df.index.min()}. Last Trading Day: {df.index.max()}')
    symbols_neg_div_df = run_algo(df)
    return symbols_neg_div_df

def generate_defi_neg_div_score(data_df, start_date):
    close_df = get_symbols_close_data_df(data_df)
    close_df = close_df[close_df.index < start_date].copy()
    symbols_neg_div_df = run_backtest(close_df)
    date = datetime(start_date.year, start_date.month, start_date.day)
    insert_defi_neg_div_score(symbols_neg_div_df, date)
    
    


In [ ]:
from datetime import timedelta
def generate_neg_div_score(df):
    now = datetime.now()
    start_date =  datetime(now.year, now.month, now.day - 3)
    while start_date < now:
        tmp_df = df[df['date'] < start_date].copy()
        generate_defi_neg_div_score(tmp_df, start_date)
        start_date = datetime(start_date.year, start_date.month, start_date.day) + timedelta(days=1)
    

In [ ]:
def merge_neg_div_score(df):
    generate_neg_div_score(df)
    neg_div_score_df = pd.read_sql_query('select * from defi_neg_div_score', __connect__())
    neg_div_score_df['timestamp_dt'] = pd.to_datetime(neg_div_score_df['timestamp_dt'])
    if 'price' in neg_div_score_df.columns:
        del neg_div_score_df['price']
    tmp_df_2 = pd.merge(df, neg_div_score_df, how = 'left', left_on=['date', 'Symbol', 'Name'], right_on = ['timestamp_dt','symbol','name'])
    del tmp_df_2['symbol']
    del tmp_df_2['name']
    del tmp_df_2['timestamp_dt']
    return tmp_df_2
    

In [ ]:
# df = merge_neg_div_score(df)

In [ ]:
# df.columns.tolist()

In [ ]:
df

## Farms And Pool

### Append the most stable and non stable farm and pool rates.

In [ ]:
stable_coins = "USDT, USDC, DAI, BUSD, SUSD, UST, TUSD".split(",")


def get_stable_coin(pair1, pair2):
    if pair1 in stable_coins:
        return pair1
    elif pair2 in stable_coins:
        return pair2
    else:
        return'NA'


def get_pancakswap_and_raydium_farm_data():
    pancakswap_farm_df = pd.read_sql_query('select * from pancakswap_farm_pool_data;', __connect__())
    pancakswap_farm_df = pancakswap_farm_df[['timestamp_dt', 'pair1', 'pair2', 'pool_rate', 'farm_payment', 'where', 'includes_stablecoin']]
    raydium_farm_df = pd.read_sql_query('select * from raydium_farm_pool_data;', __connect__())
    merged_df = pancakswap_farm_df.append(raydium_farm_df)
    return merged_df


def get_cmc_and_liqudityfolio_pool_and_farm_data_df():
    cmc_farm_df = pd.read_sql_query('select * from cmc_farming_yield;', __connect__())
    cmc_farm_df = cmc_farm_df[['timestamp_dt', 'pair1', 'pair2', 'pool_rate', 'farm_payment', 'where', 'includes_stablecoin']]
    cmc_farm_df = cmc_farm_df[cmc_farm_df['where'].isin(['PancakeSwap', 'SushiSwap', 'BakeryToken', 'SUN'])]
    liquidityfolio_pool_df = pd.read_sql_query('select * from liquidityfolio_pool;', __connect__())
    liquidityfolio_pool_df = liquidityfolio_pool_df[liquidityfolio_pool_df['where'] == 'Uniswap']
    liquidityfolio_pool_df = liquidityfolio_pool_df[['timestamp_dt', 'pair1', 'pair2', 'pool_rate', 'farm_payment', 'where', 'includes_stablecoin']]
    liquidityfolio_pool_df['pool_rate'] = liquidityfolio_pool_df['pool_rate'] * 1200
    merged_df = cmc_farm_df.append(liquidityfolio_pool_df)
    return merged_df


def get_farm_and_pool_merged_data_df():
    cmc_and_liqudityfolio_pool_and_farm_data_df = get_cmc_and_liqudityfolio_pool_and_farm_data_df()
    pancakswap_and_raydium_farm_df = get_pancakswap_and_raydium_farm_data()
    merged_df = cmc_and_liqudityfolio_pool_and_farm_data_df.append(pancakswap_and_raydium_farm_df)
    merged_df['pool_rate'] = merged_df['pool_rate']/100
    return merged_df
    

def get_pool_rates(merged_eth_df, stable, defi_symbol):
    data = [np.nan] * 4
    merged_eth_df = merged_eth_df[(merged_eth_df['includes_stablecoin'] == stable) & (merged_eth_df['farm_payment'].isnull())]
    if not merged_eth_df.empty:
        scenario_1 = merged_eth_df.sort_values(['pool_rate']).iloc[-1]
#         print([defi_symbol, merged_eth_df.shape])
        stable_coin = get_stable_coin(scenario_1['pair1'], scenario_1['pair2'])
        other_pair = scenario_1['pair2'] if scenario_1['pair1'] == defi_symbol else scenario_1['pair1']
        data = [other_pair, scenario_1['pool_rate'], stable_coin, scenario_1['where']]
    return data


def get_farm_rates(merged_eth_df, stable, defi_symbol):
    data = [np.nan] * 5
    merged_eth_df = merged_eth_df[(merged_eth_df['includes_stablecoin'] == stable) & (merged_eth_df['farm_payment'].notnull())]
    if not merged_eth_df.empty:
        scenario_1 = merged_eth_df.sort_values(['pool_rate']).iloc[-1]
        stable_coin = get_stable_coin(scenario_1['pair1'], scenario_1['pair2'])
        other_pair = scenario_1['pair2'] if scenario_1['pair1'] == defi_symbol else scenario_1['pair1']
        data = [other_pair, scenario_1['pool_rate'], stable_coin, scenario_1['where'], scenario_1['farm_payment']]
    return data


def get_farm_and_pool_rates(merged_df):
    data = [np.nan] * 18
    defi_symbol = merged_df['defi_symbol'].max()
    merged_eth_df = merged_df[(merged_df['pair1'] == defi_symbol) | (merged_df['pair2'] == defi_symbol)].copy().reset_index(drop=True)
    if not merged_eth_df.empty:
        scenario_1_data = get_pool_rates(merged_eth_df, True, defi_symbol)
        scenario_2_data = get_pool_rates(merged_eth_df, False, defi_symbol)
        scenario_3_data = get_farm_rates(merged_eth_df, True, defi_symbol)
        scenario_4_data = get_farm_rates(merged_eth_df, False, defi_symbol)
        data = []
        data.extend(scenario_1_data)
        data.extend(scenario_2_data)
        data.extend(scenario_3_data)
        data.extend(scenario_4_data)
    r_data = [defi_symbol]
    r_data.extend(data)
    return r_data


def get_stable_non_stable_pool_rates(merged_eth_df):
    grouped_df = merged_eth_df.sort_values('pool_rate').reset_index(drop=True).groupby('timestamp_dt')
    nd_list = grouped_df.apply(lambda x: get_farm_and_pool_rates(x))
    if len(nd_list) > 0:
        index = nd_list.index[0]
        data = [index]
        data.extend(nd_list[0])
        return data
    return None


def get_farm_and_pool_data_symbols(merged_df, defi_symbols):
    data = []
    for defi_symbol in defi_symbols:
        merged_eth_df = merged_df[(merged_df['pair1'] == defi_symbol) | (merged_df['pair2'] == defi_symbol)].copy().reset_index(drop=True)
        merged_eth_df['defi_symbol'] = defi_symbol
        tmp = get_stable_non_stable_pool_rates(merged_eth_df)
        if tmp:
            data.append(tmp)
    return data


def merge_farm_and_pool_data(data_df):
    defi_symbols = data_df['Symbol'].unique().tolist()
    merged_df = get_farm_and_pool_merged_data_df()
    farm_and_pool_data = get_farm_and_pool_data_symbols(merged_df, defi_symbols)
    columns = ['date', 'symbol', 'top_stable_pool_pair', 'top_stable_pool_rate', 'top_stable_pool_coin', 'top_stable_pool_where', 'top_nonstable_pool_pair', 'top_nonstable_pool_rate', 'top_nonstable_pool_coin', 'top_nonstable_pool_where', 'top_stable_farm_pair', 'top_stable_farm_rate', 'top_stable_farm_coin', 'top_stable_farm_where', 'top_stable_farm_reward', 'top_nonstable_farm_pair', 'top_nonstable_farm_rate', 'top_nonstable_farm_coin', 'top_nonstable_farm_where', 'top_nonstable_farm_reward']
    farm_and_pool_df = pd.DataFrame(farm_and_pool_data, columns=columns)
    tmp_df_4 = pd.merge(data_df, farm_and_pool_df, how='left', left_on=['date', 'Symbol'], right_on=['date', 'symbol'])
    del tmp_df_4['symbol']
    tmp_df_5 = tmp_df_4.set_index(tmp_df_4['date'])
    columns = ['top_stable_pool_pair', 'top_stable_pool_rate', 'top_stable_pool_coin', 'top_stable_pool_where', 'top_nonstable_pool_pair', 'top_nonstable_pool_rate', 'top_nonstable_pool_coin', 'top_nonstable_pool_where', 'top_stable_farm_pair', 'top_stable_farm_rate', 'top_stable_farm_coin', 'top_stable_farm_where', 'top_stable_farm_reward', 'top_nonstable_farm_pair', 'top_nonstable_farm_rate', 'top_nonstable_farm_coin', 'top_nonstable_farm_where', 'top_nonstable_farm_reward']
    for column in columns:
        tmp_df_5[column] = tmp_df_5.groupby('Symbol')[column].transform(lambda x: x.ffill())
    tmp_df_5.reset_index(drop=True)
    return tmp_df_5


In [ ]:
merged_df_ = get_farm_and_pool_merged_data_df()
merged_df_[merged_df_['where'] == 'balancer']

In [ ]:
df = merge_farm_and_pool_data(df)
df[df['date'] == pd.to_datetime("4/29/2021")]

In [ ]:
df[df['date'] == pd.to_datetime("5/1/2021")]['top_nonstable_farm_pair'].unique()

In [ ]:
# df.columns.tolist()

## Saving to csv File

In [ ]:
df.shape

In [ ]:
df.to_csv('coingecko_data_ratios_and_percentages.csv', index = False)

In [ ]:
df.date.max()

In [ ]:
df[df['Symbol'] == 'AAVE'][['Users life-to-date duneanlytics','date']]

In [ ]:
df

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df = df.sort_values(['date','Symbol', 'Name'])

(df['Symbol'] + "____" + df['Name']).unique().tolist()

In [ ]:
df[df['Symbol'] == 'BASK']

# QA

In [ ]:
def get_all_mapping_symbols(mapping_table_name, data_table_name, mapping_source_col_name, data_table_col_name):
    sql = f'select * from {mapping_table_name} where symbol != "" and {mapping_source_col_name} in (select distinct {data_table_col_name} from {data_table_name});'
#     print(sql)
    symbols = pd.read_sql_query(sql, __connect__())
    return symbols

def get_datasource_all_tracked_coin_names(data_table_name, data_table_col_name):
    sql = f'select distinct {data_table_col_name} from {data_table_name};'
#     print(sql)
    symbols = pd.read_sql_query(sql, __connect__())
    return symbols[data_table_col_name].values.tolist()

def qa_data_source(df, mapping_table_name, data_table_name, mapping_source_col_name, data_table_col_name):
    source_all_coin_names = get_datasource_all_tracked_coin_names(data_table_name, data_table_col_name)
    print(source_all_coin_names)
    print(len(source_all_coin_names))
    mapping_symbol = get_all_mapping_symbols(mapping_table_name, data_table_name, mapping_source_col_name, data_table_col_name)
    print("Total Mappings: ", mapping_symbol.shape[0])
    name_in_cmc = mapping_symbol['name_in_cmc'].values.tolist()
    captured_names_df = df[df['Name'].isin(name_in_cmc)]
    captured_Names = captured_names_df['Name'].unique().tolist()
    print("Captured Names", len(captured_Names))
    uncaptured_names = [x for x in name_in_cmc if x not in captured_Names]
    print("Uncaptured Names: ", uncaptured_names)
    print(mapping_symbol[mapping_symbol['name_in_cmc'].isin(uncaptured_names)])
    data_symbol_group = captured_names_df.groupby(['Symbol'])
    data_df = data_symbol_group.agg({'date':['max', 'min']}).reset_index()
    print(data_df)
    print("Max Date", data_df['date']['max'].unique())
    

In [ ]:
qa_data_source(df, 'tt_symbols', 'tt_coin_metrics_v2', 'name_in_tt', 'name')

In [ ]:
qa_data_source(df, 'debank_symbols3', 'debank_project_tvl', 'name_in_debank', 'project')

In [ ]:
qa_data_source(df, 'debank_symbols3', 'debank_project_dex', 'name_in_debank', 'project')

In [ ]:
qa_data_source(df, 'debank_symbols3', 'debank_project_lendings', 'name_in_debank', 'symbol')

In [ ]:
qa_data_source(df, 'defi_symbols', 'defi_project_tvl', 'name_in_defi', 'project')

In [ ]:
qa_data_source(df, 'defistation_symbols', 'defistation_data', 'name_in_defistation', 'name')

In [ ]:
qa_data_source(df, 'dune_symbols', 'dune_total_users_analysis2', 'name_in_duneanalytics', 'symbol')

In [ ]:
qa_data_source(df, 'solana_symbols', 'solanaprojects_data', 'name_in_solana', 'name')

In [ ]:
df[df['Symbol'] == 'NMR']